In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, Analisys

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
Env = Environment(
    railLength=5.2,
    latitude=32.990254,
    longitude=-106.974998,
    elevation=1400
) 

import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

Pro75M1670 = SolidMotor(
    thrustSource="data/motors/Cesaroni_M1670.eng",
    burnOut=3.9,
    grainNumber=5,
    grainSeparation=5/1000,
    grainDensity=1815,
    grainOuterRadius=33/1000,
    grainInitialInnerRadius=15/1000,
    grainInitialHeight=120/1000,
    nozzleRadius=33/1000,
    throatRadius=11/1000,
    interpolationMethod='linear'
)

Calisto = Rocket(
    motor=Pro75M1670,
    radius=127/2000,
    mass=19.197-2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    distanceRocketNozzle=-1.255,
    distanceRocketPropellant=-0.85704,
    powerOffDrag='data/calisto/powerOffDragCurve.csv',
    powerOnDrag='data/calisto/powerOnDragCurve.csv'
)

Calisto.setRailButtons([0.2, -0.5])

NoseCone = Calisto.addNose(length=0.55829, kind="vonKarman", distanceToCM=0.71971)

FinSet = Calisto.addFins(4, span=0.100, rootChord=0.120, tipChord=0.040, distanceToCM=-1.04956)

Tail = Calisto.addTail(topRadius=0.0635, bottomRadius=0.0435, length=0.060, distanceToCM=-1.194656)

def drogueTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False

def mainTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 800 m.
    return True if y[5] < 0 and y[2] < 800 else False

Main = Calisto.addParachute('Main',
                            CdS=10.0,
                            trigger=mainTrigger, 
                            samplingRate=105,
                            lag=1.5,
                            noise=(0, 8.3, 0.5))

Drogue = Calisto.addParachute('Drogue',
                              CdS=1.0,
                              trigger=drogueTrigger, 
                              samplingRate=105,
                              lag=1.5,
                              noise=(0, 8.3, 0.5))

In [ ]:
Env.info()
Pro75M1670.info()
Calisto.info()

In [ ]:
Ana = Analisys(Env, Calisto)
Ana.Apogee_By_RocketMass(5,30,50)

In [ ]:
# Aqui colocamos o valor nominal seguido do desvio padrão
# "ATUALIZADA INSPECAO" significa que o valor foi conferido e está pronto para lançamento

analysis_parameters = {
    # Propulsion Details
    "impulse": (1415.15, 35.3),                  # ATUALIZADA INSPECAO      
    "burnOut": (5.3, 1),                         # ATUALIZADA INSPECAO
    "nozzleRadius": (0.0215, 0.0004),            # ATUALIZADA INSPECAO
    "throatRadius": (0.008, 0.00005),            # ATUALIZADA INSPECAO
    "grainSeparation": (0.006, 0.0005),          # ATUALIZADA INSPECAO
    "grainDensity": (1700, 101),                 # ATUALIZADA INSPECAO
    "grainOuterRadius": (0.021, 0.001),          # ATUALIZADA INSPECAO
    "grainInitialInnerRadius": (0.009, 0.0006),  # ATUALIZADA INSPECAO
    "grainInitialHeight": (0.12, 0.0005),        # ATUALIZADA INSPECAO
    "m_prop": (2.307, 0.020),                    # ATUALIZADA INSPECAO          #Atenção, deve ser SEM a massa do propelente
    # Aerodynamic Details
    "m_aero": (1.325, 0.002),                    # ATUALIZADA INSPECAO
    "inertiaI": (2.326,0.01*2.326),              # ATUALIZADA INSPECAO (Medido em CAD)   # Obtida do CAD com o foguete DESCARREGADO
    "inertiaZ": (0.010,0.01*0.010),              # ATUALIZADA INSPECAO (Medido em CAD)   # Obtida do CAD com o foguete DESCARREGADO
    "radius": (80.95/2000,0.001),                # ATUALIZADA INSPECAO
    "distanceRocketNozzle": (-0.8626,0.003),     # REVER
    "distanceRocketPropellant": (-0.4139,0.003), # PEDENTE AERO
    "powerOffDrag": (1,0.03),                    # ATUALIZADA INSPECAO
    "powerOnDrag": (1,0.03),                     # ATUALIZADA INSPECAO
    "noseLength": (0.160, 0.0005),               # ATUALIZADA INSPECAO
    "noseDistanceToCM": (1.083, 0.0005),         # ATUALIZADA INSPECAO
    "finSpan": (0.073, 0.001),                   # ATUALIZADA INSPECAO
    "finRootChord": (0.085, 0.001),              # ATUALIZADA INSPECAO
    "finTipChord": (0.0335, 0.001),              # ATUALIZADA INSPECAO
    "finDistanceToCM": (-0.762, 0.003),          # ATUALIZADA INSPECAO    
    # Launch and Environment Details
    "windX":(1, 0.03),                           # NÃO MUDAR, PORÉM BUSCAR JUSTIFICATIVAS
    "windY":(1, 0.03),                           # NÃO MUDAR, PORÉM BUSCAR JUSTIFICATIVAS
    "inclination": (85, 1),                      # AINDA É POSSÍVEL AUMENTAR A PRECISÃO
    "heading": (199.98361111, 5),                 # Azimute calculado através de Topografia 
    "railLength": (2.2 - 0.056 - 0.923, 0.0005), # ATUALIZADA INSPECAO
    "time":[0,6,12,18],                          # NÃO MUDAR, PORÉM BUSCAR JUSTIFICATIVAS
    "day":[13,14,15,16,17,18,19,20],             # NÃO MUDAR, PORÉM BUSCAR JUSTIFICATIVAS
    "year":[2018,2019,2020],                     # NÃO MUDAR, PORÉM BUSCAR JUSTIFICATIVAS
    # Parachute Details
    "m_rec": (1.90625, 0.002),                   # Passará a ser a massa do módulo Ogiva + módulo Rec
    "CdSDrogue": (0.23, 0.069),                  # PENDENTE REC
    "CdSMain": (1.08, 0.29),                     # PENDENTE REC
    "lag_rec": (1.5, 0.2),                       # PEDENTE REC
    # Electronic Systems Details
    "m_se": (1.1185, 0.002),                     # Passará a ser a massa do módulo SE medida na integração
    "lag_se": (0, 0)}                            # PEDENTE SE

print("Dados Salvos, prossiga!")

In [ ]:
Ana.dispertion(300, analysis_parameters, "Calisto", "Calisto_dispertion", thrustSource="data/motors/Cesaroni_M1670.eng", main=True)

In [ ]:
Ana.exportElipsesToKML("Panda_Balistico")